# CS413 Image and Video Analysis -- Coursework

# Sam Moon - 2001779

# Part 3: Pick-and-Place

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import scipy.ndimage as morpho

from google.colab import drive
drive.mount('/content/drive')

data_dir = 'drive/My Drive/Colab Notebooks/data/courseworkdata/data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
def showImages (ims, cmap = None):
  fig = plt.figure(figsize=(20, 20))

  for i in range(len(ims)):
    plt.subplot(1, len(ims), i+1)
    plt.axis('off')
    plt.title("Image " + str(i))
    plt.imshow(ims[i], cmap)

  plt.show()

In [11]:
comp_names = ['BUCK', 'CAP1', 'CAP2', 'CON', 'DAC', 'DIODE', 'ENET', 'OSC1', 'OSC2', 'RECT', 'REGULATOR', 'RESISTOR', 'TRANSISTOR']

components = []
for comp in comp_names:
  one_comp = []
  for i in range(1,4):
    im = cv2.resize(plt.imread(data_dir + 'COMPONENTS/' + comp + '-' + str(i) + '.jpg')[:,:,:3], (0, 0), fx = 0.5, fy = 0.5)
    im = im[im.shape[0]//7:-im.shape[0]//7, im.shape[1]//5:-im.shape[1]//5]
    one_comp.append(im)
  showImages(one_comp)
  components.append(one_comp)

Output hidden; open in https://colab.research.google.com to view.

In [12]:
sift = cv2.SIFT_create()

component_kp = []
component_kp_des = []
for comp in components:
  fig = plt.figure(figsize=(30,30))
  j = 1
  kp = []
  des = []
  for one_comp in comp:
    one_comp_kp, one_comp_des = sift.detectAndCompute(cv2.blur(one_comp, (3, 3)), None)
    one_comp_with_kp = cv2.drawKeypoints(one_comp, one_comp_kp, outImage=np.array([]), color=(255, 0, 0))

    plt.subplot(1, 3, j)
    plt.axis('off')
    plt.imshow(one_comp_with_kp)
    j += 1

    kp.append(one_comp_kp)
    des.append(one_comp_des)
  component_kp.append(kp)
  component_kp_des.append(des)
  plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [13]:
board = plt.imread(data_dir + 'BOARDS/' + 'BOARD3-1.jpg')[:,:,:3]
board = cv2.resize(board[2*board.shape[0]//7:-2*board.shape[0]//7, 2*board.shape[1]//9:-board.shape[1]//9], (0,0), fx = 2, fy = 2)

fig = plt.figure(figsize=(30, 30))
plt.imshow(board)
plt.axis('off')
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [14]:
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)

flann = cv2.FlannBasedMatcher(index_params,search_params)

for size in [250, 200, 100]:
  for x in range(2*size, board.shape[0]-2*size, size//2):
    for y in range(2*size, board.shape[1]-2*size, size//2):
      window = board[x-size:x+size, y-size:y+size]
      window_kp, window_des = sift.detectAndCompute(window, None)
      if isinstance(window_des,type(np.array([]))):
        for i in range(len(components)):
          for j in range(len(components[i])):
            matches = flann.knnMatch(component_kp_des[i][j],window_des,k=2)

            matchesMask = [[0,0] for i in range(len(matches))]

            inliers = 0
            for k,(m,n) in enumerate(matches):
                if m.distance < 0.4*n.distance:
                    matchesMask[k]=[1,0]
                    inliers += 1

            draw_params = dict(singlePointColor = (255,120,0),
                               matchesMask = matchesMask,
                               flags = cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS+
                                       cv2.DrawMatchesFlags_DRAW_RICH_KEYPOINTS)

            if inliers > 4:
              component_matches_im = cv2.drawMatchesKnn(components[i][j],component_kp[i][j],window,window_kp,matches,None,**draw_params)
              fig = plt.figure(figsize=(30,30))
              plt.axis('off')
              plt.imshow(component_matches_im)
              plt.show()

Output hidden; open in https://colab.research.google.com to view.